In [0]:
import mlflow

In [0]:
pip install lightgbm

In [0]:
%restart_python

In [0]:

import mlflow
import mlflow.lightgbm
import pyspark.sql.functions as F

In [0]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [0]:

model_name = "ispl_databricks.model_logs.final_bd_model"

challenger_version = client.get_model_version_by_alias(
    name=model_name,
    alias="Challenger"
).version

print("Challenger model version:", challenger_version)

In [0]:
versions = client.search_model_versions(f"name='{model_name}'")

latest_version = max(int(v.version) for v in versions)

In [0]:
model_info = mlflow.models.get_model_info(f"models:/{model_name}/{latest_version}")

In [0]:

feature_names = model_info.signature.inputs.input_names()

In [0]:
latest_version

In [0]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{latest_version}")

In [0]:
features = model.metadata.get_input_schema().inputs

In [0]:
spark_df = spark.table('ispl_databricks.model_logs.bd_final_feature_stores')

In [0]:
spark_df = spark_df.select(feature_names)

In [0]:
table_name = f"ispl_databricks.model_logs.bd_champion_{latest_version}"

In [0]:
spark_df.write \
    .format("delta") \
    .mode("append") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_name)

In [0]:
client.set_registered_model_alias(
    name=model_name,
    alias="Champion",
    version=challenger_version
)